In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2020-09-11 23:48:30--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.60.72.32, 2600:1409:12:285::e59, 2600:1409:12:2af::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.60.72.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   118MB/s    in 6.7s    

2020-09-11 23:48:37 (118 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [ ]:
!unzip kagglecatsanddogs_3367a.zip

In [ ]:
dataset_path = '/content/PetImages'

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Deleted {num_skipped} images because not worth it")

Deleted 0 images because not worth it


In [ ]:
image_size = (224, 224)
batch_size = 64

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed = 1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed =1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [ ]:
# Data Augmentation Layer for training

data_augmentation_train = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.Rescaling(scale =1./255),
        layers.experimental.preprocessing.RandomHeight(0.1),
        layers.experimental.preprocessing.RandomWidth(0.1)
     
    ]
)

# Data Augmentation Layer for testing

data_augmentation_test = keras.Sequential(
    [
        layers.experimental.preprocessing.Rescaling(scale =1./255)
     
    ]
)

In [ ]:
# Map the Data now

augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation_train(x, training=True), y))

augmented_val_ds = val_ds.map(
    lambda x, y: (data_augmentation_test(x, training=True), y))

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape=(224,224,3), include_top=False, weights='imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import regularizers

In [ ]:
global_average_layers = tf.keras.layers.GlobalAveragePooling2D()
dropouts = tf.keras.layers.Dropout(rate = 0.2)

dense_128 = tf.keras.layers.Dense(128)
dense_64 = tf.keras.layers.Dense(64)
dense_32 = tf.keras.layers.Dense(32)

prediction_layer = tf.keras.layers.Dense(1,activation='sigmoid')

model_V3 = tf.keras.Sequential([
    pre_trained_model,
    global_average_layers,
    dropouts,
    dense_128,
    dense_64,
    dropouts,
    dense_32,
    prediction_layer
])

model_V3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_V3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [ ]:
# Callbacks
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

filepath = "catvsdogsmodel.h5"

tensorboard_callback = tf.keras.callbacks.TensorBoard("logs")
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callback_list = [tensorboard_callback, lr_reduce, checkpoint, early_stop]

In [ ]:
delete_logs = False

if delete_logs:
    import shutil
    shutil.rmtree('logs')
else:
    print('To delete model logs change delete_logs to True.')
    raise UnboundLocalError

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
history = model_V3.fit(
           augmented_train_ds,
           epochs=6, validation_data=augmented_val_ds, callbacks=callback_list)

Epoch 1/6
293/293 [==============================] - ETA: 0s - loss: 0.1275 - accuracy: 0.9627
Epoch 00001: val_loss improved from inf to 0.03004, saving model to catvsdogsmodel.h5
293/293 [==============================] - 234s 800ms/step - loss: 0.1275 - accuracy: 0.9627 - val_loss: 0.0300 - val_accuracy: 0.9895
Epoch 2/6
293/293 [==============================] - ETA: 0s - loss: 0.0693 - accuracy: 0.9749
Epoch 00002: val_loss did not improve from 0.03004
293/293 [==============================] - 202s 689ms/step - loss: 0.0693 - accuracy: 0.9749 - val_loss: 0.0342 - val_accuracy: 0.9891
